In [1]:
list(filter(lambda d: d is 1, [1,2,3,4,5]))

<>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
/tmp/ipykernel_49792/1338367584.py:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
  list(filter(lambda d: d is 1, [1,2,3,4,5]))


[1]

In [2]:
from compile_data import parse_drawio_as_xml, extract_shape_objects_and_edge_mxcells, extract_shape_params, extract_edge_params

ImportError: attempted relative import with no known parent package

In [ ]:
import xml.etree.ElementTree as ET

xml: ET.ElementTree = parse_drawio_as_xml('drawio_dataset/driving school 2-技4.drawio')
shape_objs, edge_mxcells = extract_shape_objects_and_edge_mxcells(xml)

shape_ids, shape_titles, positions, dimensions = extract_shape_params(shape_objs)
edge_styles, source_ids, target_ids, edge_points = extract_edge_params(edge_mxcells, shape_objs)

['cropped_image_6.jpeg', 'cropped_image_5.jpeg', 'YFMHQ4FhG7uMwye2RJbG-13', 'cropped_image_3.jpeg', 'cropped_image_4.jpeg', 'cropped_image_2.jpeg', 'cropped_image_7.jpeg', 'cropped_image_1.jpeg', 'cropped_image_0.jpeg', None, None, None, None, None, None, None, None, None, 'cropped_image_8.jpeg', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'A1938f57c7289d4bea7031043781a408c8c', 'A1499ac42926efa5769701bba51dd52f934', 'A11bd3f7aa00195ccc74d42741409abea41', 'A5e83e9d6cf87761df3cd24bb4ccd85a41', 'A224494be28a13b819e428d8a1c4e8808e', 'A3dd60eda646ebb0da503ead92d8a12160', 'A40706dd8bd651bb5ddc0644ad07302675', 'A9504ffbcd7184f90bf5125bcc4feffef0', 'A6d93905f8f6a5f78cea5cbc225087047d', 'A701e505cec0eb7998b223d53a8b7408d0', 'A80bdf8b8770bd92308a420cdbc7df022d', 'A100681994967977c00d64b8d6c180225e0', 'A12c6682aa4fc534eed205ff1a56a20ecd6', 'A132757e1dff67c3

In [ ]:
import json
with open('graph_dataset.json', 'r') as handle:
    data = json.load(handle)

In [ ]:
from typing import List, NamedTuple, Tuple


class Graph(NamedTuple): 
    titles: List[str]
    positions: List[Tuple[float, float]]
    dimensions: List[Tuple[float, float]]
    adjacency_matrix: List[List[int]]

In [ ]:
graphs: List[Graph] = [Graph(*d) for d in data]

In [ ]:
graphs[0]

Graph(titles=['<font style="font-size: 14px;">Research Workbook Concept</font>', '<font style="font-size: 14px;">Intent</font>', '<font style="font-size: 14px;">Make a structured procedure that can be followed during research such that:<br><br>* You know what needs to be done next<br>* Keep track of and manage uncertainties<br>* Identify how much work is left<br>* Identify if you are on schedule<br></font>', '<font style="font-size: 14px;">Research Plan = Stratgic Plan + Implementation Plan Concept</font>', '<span style="font-size: 14px;">Structure of the Research Plan</span>', '<font style="font-size: 14px;">Research Plan</font>', '<font style="font-size: 14px;">Strategic Plan</font>', '<font style="font-size: 14px;">Implementation Plan</font>', '<font style="font-size: 14px;">Recon</font>', '<font style="font-size: 14px;">Strategic Concepts</font>', '<font style="font-size: 14px;">Strategic Goals</font>', '<font style="font-size: 14px;">Macroscopic View</font>', '<font style="font-si

In [ ]:
from torch import FloatTensor
FloatTensor([1])

tensor([1.])

In [ ]:
# PositionalEncoding implementation from https://pytorch.org/tutorials/beginner/transformer_tutorial.html#define-the-model
def positional_encoding(positions: FloatTensor, d_model=50):
    div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
    
    pe = torch.zeros(positions.shape[0], d_model)
    positions = positions.unsqueeze(-1)
    pe[:, 0::2] = torch.sin(positions * div_term)
    pe[:, 1::2] = torch.cos(positions * div_term)
    return pe

In [ ]:
positional_encoding(FloatTensor([1])).shape

torch.Size([1, 50])

In [ ]:
from sentence_transformers import SentenceTransformer
import torch
from torch import nn
from torch import FloatTensor

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
embedding = PositionalEncoding(50, max_len=5440)
def vectorise_nodes(graph: Graph, sent_bert):
    ''' Converts each nodes in the graph into a vector of shape (num_nodes, 584).
        (384, ) sentence embedding
        (50, )  Position Embedding of x position of shape
        (50, )  Position Embedding of y position of shape
        (50, )  Position Embedding of width of shape
        (50, )  Position Embedding of height of shape
    '''
    graph.titles
    text_encoding = sent_bert.encode(graph.titles)
    x_encoding = positional_encoding(FloatTensor([position[0] for position in graph.positions]))
    y_encoding = positional_encoding(FloatTensor([position[1] for position in graph.positions]))
    width_encoding = positional_encoding(FloatTensor([dim[0] for dim in graph.dimensions]))
    height_encoding = positional_encoding(FloatTensor([dim[1] for dim in graph.dimensions]))
    return torch.hstack([torch.Tensor(text_encoding), x_encoding, y_encoding, width_encoding, height_encoding])
    

In [ ]:
node_vectors = vectorise_nodes(graphs[0], model)

In [ ]:
node_vectors.shape

torch.Size([100, 584])

In [ ]:
max_value = 0
min_value = 0
for graph in graphs:
    max_value = max(graph.positions[0]+[max_value])
    min_value = min(graph.positions[0]+[min_value])

In [ ]:
print(max_value, min_value)

4550.0 -870.0


In [ ]:
for graph in graphs:
    node_vectors = vectorise_nodes(graph, model, embedding)

TypeError: vectorise_nodes() takes 2 positional arguments but 3 were given

In [ ]:
from torch_geometric.data import Data

In [ ]:
Data(x=node_vectors, y=node_vectors, edge_index=graphs[0].adjacency_matrix)

Data(x=[100, 584], edge_index=[100], y=[100, 584])

In [ ]:
from torch_geometric.data import InMemoryDataset, download_url


class MyOwnDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.load(self.processed_paths[0])
        
    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        # return ['data.pt']
        return ['data.pt']

    def process(self):
        # Read data into huge `Data` list.
        with open('graph_dataset.json', 'r') as handle:
            json_data = json.load(handle)
        graphs: List[Graph] = [Graph(*d) for d in json_data]
        data_list: List[Data] = []
        for graph in graphs:
            node_vectors = vectorise_nodes(graph, model)
            data: Data = Data(x=node_vectors, y=node_vectors, edge_index=graph.adjacency_matrix)
            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        self.save(data_list, self.processed_paths[0])
        # For PyG<2.4:
        # torch.save(self.collate(data_list), self.processed_paths[0])

In [ ]:
import torch_geometric.transforms as T
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'])

dataset[0]

Extracting /tmp/ShapeNet/shapenetcore_partanno_segmentation_benchmark_v0_normal.zip
Processing...
Done!


Data(x=[2518, 3], y=[2518], pos=[2518, 3], category=[1])

In [ ]:
dataset[0].pos

tensor([[-0.0145, -0.0164,  0.0320],
        [-0.0119, -0.0657,  0.0145],
        [-0.1424, -0.0370, -0.0519],
        ...,
        [ 0.0342, -0.0931, -0.0523],
        [-0.0108, -0.0600,  0.0522],
        [-0.0165, -0.0593,  0.0560]])

In [ ]:
d = MyOwnDataset('/home/sp/Downloads/mindmap_ai/')

!!!
/home/sp/Downloads/mindmap_ai/processed/data.pt
!!!


In [ ]:
d[2].x

tensor([[-0.0783,  0.4326,  0.0079,  ...,  0.9999,  0.0116,  0.9999],
        [-0.1276,  0.5939,  0.0324,  ...,  0.9999,  0.0116,  0.9999],
        [-0.0104, -0.2817, -0.3822,  ...,  0.9999,  0.0116,  0.9999],
        ...,
        [-0.0585, -0.3001, -0.0923,  ...,  0.9999,  0.0072,  1.0000],
        [-0.0181, -0.2946,  0.0435,  ...,  0.9999,  0.0072,  1.0000],
        [-0.5034,  0.0038,  0.2237,  ...,  0.9999,  0.0087,  1.0000]])

In [ ]:
print(d[0])

Data(x=[100, 584], edge_index=[100], y=[100, 584])


In [ ]:
d[0]

Data(x=[100, 584], edge_index=[100], y=[100, 584])

In [ ]:
a = torch.LongTensor([[0,1], [0, 1]])
b = torch.LongTensor([[2,3], [2, 3]])


In [ ]:
torch.hstack([a,b])

tensor([[0, 1, 2, 3],
        [0, 1, 2, 3]])

In [ ]:
embedding(torch.LongTensor([1]))

tensor([[-0.0739, -1.3733,  0.6383,  1.2986, -0.5680, -0.6190,  2.7902, -0.9930,
          0.9808, -0.0230,  0.2395, -0.1810,  1.5876, -0.1068, -0.0474,  1.3623,
          0.0973,  0.5638, -0.1797,  1.4252,  0.2016, -0.1904,  0.3645, -0.7482,
          0.7876,  0.9746, -1.3740,  0.2264, -0.0042, -0.6097, -0.3536,  0.7618,
         -0.9047,  1.9783,  0.0971, -0.0237,  1.5586, -0.3120,  0.4342, -0.6982,
          0.1947,  0.5339, -0.0351,  0.3205, -0.2972,  0.2540,  1.9901, -1.5897,
          0.6215, -0.8268]], grad_fn=<EmbeddingBackward0>)

In [ ]:
max([d[0] for d in list(chain(*[graphs[i].positions for i in range(len(graphs))]))])

2256.0

In [ ]:
max([d[1] for d in list(chain(*[graphs[i].positions for i in range(len(graphs))]))])

5440.0

In [ ]:
embedding(torch.LongTensor([1,2,9]))

tensor([[-0.0239,  1.0657, -1.7120],
        [-1.2933, -0.4375, -1.0517],
        [ 0.1095, -1.0512, -0.0409]], grad_fn=<EmbeddingBackward0>)

In [ ]:
model(**tokenizer(graphs[0].titles, return_tensors='pt', padding=True)).last_hidden_state.shape

torch.Size([100, 83, 768])

In [ ]:
import torch

In [ ]:
embedding = nn.Embedding(5440, 10)

In [ ]:
d[0].train_mask

AttributeError: 'GlobalStorage' object has no attribute 'train_mask'

In [ ]:
from torch_geometric.loader import DataLoader

In [ ]:
[b for b in DataLoader(d)][0].train_mask

AttributeError: 'GlobalStorage' object has no attribute 'train_mask'

In [ ]:
d[0][torch.Tensor([True for i in range(len(d[0]))], dtype=torch.bool)]

TypeError: new() received an invalid combination of arguments - got (list, dtype=torch.dtype), but expected one of:
 * (*, torch.device device)
      didn't match because some of the keywords were incorrect: dtype
 * (torch.Storage storage)
 * (Tensor other)
 * (tuple of ints size, *, torch.device device)
 * (object data, *, torch.device device)


In [ ]:
from torch import BoolTensor
d[0].x[BoolTensor([True for i in range(len(d.x[0]))])]

IndexError: The shape of the mask [584] at index 0 does not match the shape of the indexed tensor [100, 584] at index 0

In [ ]:
d[0].y.shape

torch.Size([100, 584])

In [ ]:
d[0].y[:, BoolTensor([False, False,False, False]+[True for i in range(580)])].shape

torch.Size([100, 580])

In [12]:
import json
from typing import List
from models import Graph
with open('graph_dataset.json', 'r') as handle:
    json_data = json.load(handle)
graphs: List[Graph] = [Graph(*d) for d in json_data]

In [13]:
xs, ys = list(zip(*graphs[0].positions))

In [14]:
ws, hs = list(zip(*graphs[0].dimensions))

In [15]:
max(hs)

136.0

In [16]:
max(ws)

390.0

In [17]:
max(ys)

2115.0

In [18]:
min(ys)

0

In [19]:
import numpy as np
mu = np.array([1,2,3])
sigma = 1000
s = np.random.normal(np.array([0,0,0]), sigma)

In [21]:
import numpy as np
mu = np.array([1,2,3])
sigma = 200
s = np.random.normal(np.array([0,0,0]), sigma)

In [22]:
np.random.randint(low=0.0, high=100)

48

In [23]:
np.random.randint(low=0.0, high=100)

14

In [25]:
np.random.uniform(0, 1)

0.587228315524544

In [ ]:
def add_noise(data: np.array, sigma: List[int])->np.array:
    noise = np.random.normal(np.zeros(len(data)), sigma)
    scale_noise = np.random.uniform(0, 1)
    noise = scale_noise*noise
    data += noise
    return data

In [31]:
list(zip(*([1,1,1], [2,2,2])))

[(1, 2), (1, 2), (1, 2)]

In [26]:
def noise_nodes(node_titles: List[str], positions: List[Tuple[float, float]], dimensions: List[Tuple[float, float]])->FloatTensor:
    xs, ys = tuple(zip(*graphs[0].positions))
    ws, hs = tuple(zip(*graphs[0].dimensions))
    xs = add_noise(np.array(xs), 1000)
    ys = add_noise(np.array(ys), 1000)
    ws = add_noise(np.array(ws), 200)
    hs = add_noise(np.array(hs), 200)
    positions = zip(*(xs, ys))
    dimensions = zip(*(ws, hs))
    return vectorise_nodes(node_titles, positions, dimensions)
    

NameError: name 'Tuple' is not defined

In [77]:
from generate_dataset import MyOwnDataset

In [107]:
dataset = MyOwnDataset('/home/sp/Downloads/mindmap_ai/')

In [108]:
type(dataset[0].edge_index)

torch.Tensor

In [109]:
from torch_geometric.loader import DataLoader
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [110]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

In [111]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, 4)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.relu(x)

In [129]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
# data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

model.train()
# 200
for epoch in range(10000):
    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        target = data.y.to(device)
        loss = F.mse_loss(out, target)
        loss.backward()
        optimizer.step()
        print(loss)

tensor(420060.5625, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(419788.1250, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(419469.3750, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(419109.4062, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(418724.0312, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(418296.8438, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(417910.8438, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(417408.2188, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(417163.0312, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(416542.2812, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(416205.1250, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(415590.5625, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(414993.2812, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(414584.8125, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(413832.8750, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(413553.5938, devic

In [132]:
target

tensor([[ 60.0000, 110.0000, 120.0000,  28.0000],
        [110.0000, 208.0000, 390.0000,  48.0000],
        [142.5000, 410.5000, 160.0000,  27.5000],
        ...,
        [187.3600, 719.0000,  16.0000,  16.0000],
        [171.3600, 630.0000,  16.0000,  16.0000],
        [176.0000, 595.0000,  16.0000,  16.0000]], device='cuda:0')

In [133]:
out

tensor([[183.6207, 346.9236,  69.1874,  34.5349],
        [275.0488, 304.8510,  80.5564,  36.7310],
        [234.7885, 253.9378,  78.0067,  35.5578],
        ...,
        [263.0983, 934.3291, 106.2120,  41.5636],
        [137.9598, 237.4277,  70.7417,  33.3280],
        [517.3143, 785.5502, 140.5710,  49.1442]], device='cuda:0',
       grad_fn=<ReluBackward0>)

In [134]:
target.flatten()

tensor([ 60., 110., 120.,  ..., 595.,  16.,  16.], device='cuda:0')

In [97]:
torch.hstack(torch.Tensor([[1,2,3,4],[1,2,3,4]]))

TypeError: hstack(): argument 'tensors' (position 1) must be tuple of Tensors, not Tensor

In [102]:
torch.tensor(list(zip([1,2,3],[4,5,6])))

tensor([[1, 4],
        [2, 5],
        [3, 6]])

In [105]:
torch.Tensor([[1,2,3],[4,5,6]]).T

tensor([[1., 4.],
        [2., 5.],
        [3., 6.]])

In [124]:
F.mse_loss(torch.FloatTensor([[0,0,0],[0,0,0],[0,0,0]]), torch.FloatTensor([[1,1,1],[1,1,1],[1,1,1]]))

tensor(1.)

In [1]:
def parse_drawio_as_xml(drawio_file_path: str) -> ET.ElementTree:
    '''Opens a drawio file, and return an xml representation of it.'''
    diagram = drawio_diagram()
    diagram.from_file(drawio_file_path)
    xml_str: str = diagram.dump_xml()
    f = io.StringIO(xml_str)
    tree: ET.ElementTree = ET.parse(f)
    return tree

NameError: name 'ET' is not defined

In [2]:
import xml.etree.ElementTree as ET
import io

In [3]:
with open('drawio_dataset/driving school 2-技4.drawio', 'r') as handle:
    tree: ET.ElementTree = ET.parse(handle)

In [41]:
tree.find("//object[@id='A11bd3f7aa00195ccc74d42741409abea41']/mxCell").attrib['x'] = 200

/tmp/ipykernel_8187/4098921390.py:1: FutureWarning: This search is broken in 1.3 and earlier, and will be fixed in a future version.  If you rely on the current behaviour, change it to ".//object[@id='A11bd3f7aa00195ccc74d42741409abea41']/mxCell"
  tree.find("//object[@id='A11bd3f7aa00195ccc74d42741409abea41']/mxCell").attrib['x'] = 200


In [30]:
tree.findall("diagram/mxGraphModel/root/object")[10].attrib['id']

'A11bd3f7aa00195ccc74d42741409abea41'

In [36]:
tree.findall("diagram/mxGraphModel/root/object")[10].find('mxCell').attrib['x']

200

In [2]:
import torch
[torch.Tensor([1,2,3])]*3

[tensor([1., 2., 3.]), tensor([1., 2., 3.]), tensor([1., 2., 3.])]

In [10]:
torch.stack([torch.Tensor([[1,2,3], [4,5,6]])]*3)

tensor([[[1., 2., 3.],
         [4., 5., 6.]],

        [[1., 2., 3.],
         [4., 5., 6.]],

        [[1., 2., 3.],
         [4., 5., 6.]]])

In [24]:
torch.stack([torch.Tensor([[[1,2,3], [4,5,6]]]), torch.Tensor([[[0,0,0,0], [0,0,0,0]]])], dim=-1)

RuntimeError: stack expects each tensor to be equal size, but got [1, 2, 3] at entry 0 and [1, 2, 4] at entry 1

In [19]:
torch.Tensor([[1,2,3], [4,5,6]]).shape

torch.Size([2, 3])

In [27]:
torch.concat((torch.zeros((3, 512, 768)), torch.zeros((3, 512, 50))), dim=-1).shape

torch.Size([3, 512, 818])

In [28]:
768+50

818